We are importing the required libraries 

In [12]:
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense, Embedding,SpatialDropout1D
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
import numpy as np 
import pandas as pd
from keras.layers.recurrent import LSTM
from keras.preprocessing.sequence import pad_sequences

We are reading the data. In my folder, I have named it spam.csv. If you want to download the file here is the kaggle link for it.
https://www.kaggle.com/ishansoni/sms-spam-collection-dataset

In [13]:
df = pd.read_csv('spam.csv',encoding='latin-1')

In [14]:
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


Here in the head we can see there are some unnecessary columns. Let's delete them

In [15]:
df = df.drop(labels = ["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis = 1)

In [16]:
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


Now we are importing stopwords. If you are not familiar with them. This is the link for it.
https://pythonspot.com/nltk-stop-words/

In [17]:
import re
from nltk.corpus import stopwords

In [30]:
df = df.reset_index(drop=True)
replace_space = re.compile('[/(){}\[\]\|@,;]')
bad_symbols = re.compile('[^0-9a-z #+_]')
stopwords = set(stopwords.words('english'))

def clean_text(text):
    
    text = text.lower() 
    text = replace_space.sub(' ', text)
    text = bad_symbols.sub('', text) 
    text = text.replace('x', '')
    text = ' '.join(word for word in text.split() if word not in stopwords) 

lets Tokenize the data

In [31]:
Max_words = 1000
Max_sequence =50
Embedding_Dim = 100
tokenizer = Tokenizer(num_words=Max_words, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df['v2'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 8920 unique tokens.


In [21]:
X = tokenizer.texts_to_sequences(df['v2'].values)
X = pad_sequences(X, maxlen=)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (5572, 50)


In [22]:
Y = pd.get_dummies(df['v1']).values
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (5572, 2)


From sklearn lets do a training and testing split

In [24]:
import sklearn 
from sklearn.model_selection import train_test_split

In [25]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 101)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(3733, 50) (3733, 2)
(1839, 50) (1839, 2)


Lets pass a keras layers into our model. I have used embedding, dropout, LSTM and Dense layer. I have ran 30 epochs with early stopping.

In [27]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
epochs = 30
batch_size = 64
history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

W0718 09:56:25.468976 4658869696 module_wrapper.py:136] From /anaconda3/lib/python3.7/site-packages/tensorflow_core/python/util/module_wrapper.py:163: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0718 09:56:25.493772 4658869696 deprecation.py:506] From /anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0718 09:56:25.888346 4658869696 module_wrapper.py:136] From /anaconda3/lib/python3.7/site-packages/tensorflow_core/python/util/module_wrapper.py:163: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0718 09:56:25.999025 4658869696 deprecation.py:323] From /anaconda3/lib/python3.7/site-packages/tensorflow_core/python/ops/math_gra

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 100)           100000    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 50, 100)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 202       
Total params: 180,602
Trainable params: 180,602
Non-trainable params: 0
_________________________________________________________________
Train on 3359 samples, validate on 374 samples
Epoch 1/30
3359/3359 [==============================] - 4s 1ms/step - loss: 0.3054 - acc: 0.8901 - val_loss: 0.1349 - val_acc: 0.9679
Epoch 2/30
3359/3359 [==============================] - 3s 843us/step - loss: 0.0699 - 

Lets check the accuracy of our model.

In [29]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

1839/1839 [==============================] - 0s 254us/step
Test set
  Loss: 0.079
  Accuracy: 0.985


98.5 % accuracy Pretty Good huh !! Thanks guys